<a href="https://colab.research.google.com/github/Hyun-Ho95/Web1/blob/master/Mini_project__%EA%B5%AD%EB%AF%BC%EB%8F%99%EC%9D%98%EC%B2%AD%EC%9B%90_%EC%8A%A4%ED%81%AC%EB%9E%98%ED%95%91_%EB%B0%95%ED%98%84%ED%98%B8_ipynb.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 라이브러리 불러오기

In [2]:
import time
import pandas as pd
import requests
from bs4 import BeautifulSoup as bs

## 크롬 브라우저의 네트워크 정보를 통한 데이터 수집
* json 방식 데이터 수집

In [3]:
# 청원 홈페이지 url 찾기
# /onGoing -> 동의진행 청원
# /agreeEnded -> 동의종료 청원
# /referredAll -> 위원회 회부청원

In [8]:
page_index = 1
count_per_page = 8
begin_date = 20220101
end_date = 20221231

In [9]:
url = f'https://petitions.assembly.go.kr/api/petits?pageIndex={page_index}&recordCountPerPage={count_per_page}&sort=AGRE_END_DE-&searchCondition=sj&searchKeyword=&petitRealmCode=&sttusCode='
url = f'{url}PETIT_FORMATN,CMIT_FRWRD,PETIT_END&resultCode=BFE_OTHBC_WTHDRAW,PROGRS_WTHDRAW,PETIT_UNACPT,APPRVL_END_DSUSE,ETC_TRNSF&notInColumn='
url = f'{url}RESULT_CODE&beginDate={begin_date}&endDate={end_date}&ageCd='
url

'https://petitions.assembly.go.kr/api/petits?pageIndex=1&recordCountPerPage=8&sort=AGRE_END_DE-&searchCondition=sj&searchKeyword=&petitRealmCode=&sttusCode=PETIT_FORMATN,CMIT_FRWRD,PETIT_END&resultCode=BFE_OTHBC_WTHDRAW,PROGRS_WTHDRAW,PETIT_UNACPT,APPRVL_END_DSUSE,ETC_TRNSF&notInColumn=RESULT_CODE&beginDate=20220101&endDate=20221231&ageCd='

## Pandas를 통한 요청

In [23]:
# json데이터 확인해보니 청원 목록 = [{청원내용 1에 대한 데이터}, {청원내용 2에 대한 데이터}, ....]인 것 같습니다. 
# 이것저것 시도해봤는데 원하는 데이터 뽑으려면 일단 수동으로 키값은 찾아야 할 것 같습니다        
petit_df = pd.read_json(url, encoding = 'UTF-8')[['petitRealmCode', 'petitRealmNm', 'petitSj', 'petitCn', 'jrsdCmitNm', 'resultCodeNm', 'agreEndDe', 'agreCo']]
cols = ['분야코드', '분야', '청원제목', '청원내용', '심사위원회', '청원결과', '종료날짜','동의수' ]
petit_df.columns = cols
# 저는 이 정도가 데이터 분석해볼 때 필요한 항목이라고 생각해서 가져왔습니다.                                                                                        
# 시작날짜는 종료날짜에서 한달 빼면 되고 , 심사위원회는 어느 심사위원회가 심사할 때 통과가 많았는지가 궁금해서 가져와 보았습니다.
# 청원내용문 안에 저 '\r\n'문자가 지저분해서 제거하려고 하는데 잘 안 없어지네요
petit_df

,분야코드,분야,청원제목,청원내용,심사위원회,청원결과,종료날짜,동의수
0,TRIT,국토/해양/교통,"인천 미추홀구뿐만 아니라 전국적인 ‘나 홀로 아파트, 빌라’ 전세사기에 관한 청원",○ 문제점\r\n낙찰자가 생겨 곧 퇴거해야하는 피해자들을 위한 ‘전세사기피해지원센터...,국토교통위원회,동의만료폐기,2022-12-30 23:59:59,2250
1,TRIT,국토/해양/교통,"모든 학원, 상가 복도 cctv 의무화에 관한 청원",안녕하세요.\n봉담읍에 사는 초등학생 아이를 둔 엄마입니다.\n저희아이는 학원강의...,국토교통위원회,동의만료폐기,2022-12-30 23:59:59,283
2,HMCCS,보건의료,간호법 제정에 관한 청원,"현재 96개국에 '간호법'이 있고, OECD 국가 38개국 중 미국을 포함한 33개...",보건복지위원회,동의만료폐기,2022-12-30 23:59:59,177
3,HMCCS,보건의료,난치성 편두통 환자를 위한 CGRP 표적 치료제의 급여혜택 강화 촉구에 관한 청원,"건강보험심사평가원에서 공개한 고시 자료[1]를 보면 대상 질환의 특성으로서 ""삽화편...",보건복지위원회,동의만료폐기,2022-12-28 23:59:59,524
4,FNANC,재정/세제/금융/예산,가상자산 위믹스 코인 거래지원종료에 관한 DAXA의 처리 투명성과 갑질논란에 관한 청원,한국 거래소협회 DAXA는 10월27일 계획유통량과 실제유통량의 차이를 원인으로 삼...,정무위원회,동의만료폐기,2022-12-28 23:59:59,6165
5,MSFRTN,재난/안전/환경,기업의 과대포장에 관한 청원,환경 문제가 날이 갈수록 심화되고 있습니다. 세계가 노력하고 있지만 여전히 부족하며...,환경노동위원회,동의만료폐기,2022-12-28 23:59:59,157
6,HMCCS,보건의료,크론병을 앓고 있는 아들의 건강보험 적용 요청에 관한 청원,저의 아들은 크론병을 앓고 있습니다.\r\n크론병 치료나 완화에 사용되는 약 또는 ...,보건복지위원회,동의만료폐기,2022-12-28 23:59:59,650
7,FAMILY,저출산/고령화/아동/청소년/가족,난임부부지원 횟수의 확대 및 시험관 지원에 관한 청원,저는 아직 시험관을 시작하지 않은 결혼 2년차인 여성입니다.\r\n \r\n늦은 결...,보건복지위원회,동의만료폐기,2022-12-28 23:59:59,567


#분석 방향

In [6]:
# 1. 분야 별 만료폐기 건과 통과된 건 구분
# 2. 심사위원회 별 통과 건
# 3. 만료폐기 건 중 동의 수 30000이 넘는 건(다시 청원시도 하면 가능성이 있는 건)

# 이방향으로 조금 더 시도해보도록 하겠습니다..!